<a href="https://colab.research.google.com/github/keivernunez/dataminingavanzado_austral/blob/main/Clase01_Note_2_de_5_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Máquinas de Vectores de Soporte (SVM): Guía Teórica y Práctica

**Introducción:**

Este cuaderno proporciona una introducción exhaustiva a las Máquinas de Vectores de Soporte (SVM), un algoritmo de aprendizaje supervisado de gran potencia, utilizado principalmente para problemas de clasificación como también de regresión. Las SVM son particularmente efectivas en espacios de alta dimensionalidad y cuando el número de dimensiones excede el número de muestras.

El objetivo fundamental de SVM es determinar un **hiperplano** óptimo que separe las distintas clases dentro del espacio de características. En este cuaderno, se explorarán los conceptos fundamentales, se mostrará un ejemplo práctico de clasificación con el conjunto de datos del Titanic, se visualizarán las fronteras de decisión y se incluirá un ejemplo de clustering para determinar un número óptimo de agrupaciones.

## Conceptos Fundamentales de SVM

Una SVM busca un hiperplano que separe las clases con el margen máximo posible. En un problema de clasificación binaria, este hiperplano es una superficie (una línea en 2D, un plano en 3D) que divide el espacio de características.

**1. Hiperplano:** Es el plano de decisión que segrega las clases. En un problema con 2 características (2D), se trata de una línea. Con 3 características, es un plano; con más de tres, se generaliza al concepto de hiperplano.

**2. Vectores de Soporte (Support Vectors):** Corresponden a las instancias de datos de cada clase que se encuentran **más próximas** al hiperplano de decisión. Estos puntos son cruciales, ya que son los más complejos de clasificar y definen de manera unívoca la posición y orientación del hiperplano.

**3. Margen (Margin):** Es la distancia perpendicular entre el hiperplano y los vectores de soporte más cercanos. El objetivo de SVM no es simplemente separar las clases, sino hacerlo con el **margen más amplio posible (Maximum-Margin Hyperplane)**. Un margen amplio se correlaciona con una mejor capacidad de generalización del modelo, haciéndolo más robusto frente a datos no vistos.

**4. Márgenes Duros vs. Suaves (Hard vs. Soft Margins):**
    - **Margen Duro:** Impone una restricción estricta: no se permite ningún error de clasificación. Este enfoque solo es viable si los datos son perfectamente separables de forma lineal, una condición infrecuente en problemas reales.
    - **Margen Suave:** Introduce flexibilidad al permitir que algunas instancias sean clasificadas incorrectamente o queden dentro del margen. Esto es esencial para manejar datos con ruido o solapamiento entre clases. El hiperparámetro de regularización `C` controla este compromiso:
        - Un valor de `C` **elevado** impone una alta penalización a los errores, resultando en un margen más estricto y un modelo con menor sesgo pero mayor varianza (riesgo de sobreajuste).
        - Un valor de `C` **bajo** permite una mayor cantidad de errores, generando un margen más suave y un modelo con mayor sesgo pero menor varianza (riesgo de subajuste).

### El Truco del Kernel (Kernel Trick) 🧠✨

Cuando los datos no son separables mediante un hiperplano lineal en su espacio original, SVM emplea una de sus características más potentes: el **truco del kernel**.

Un *kernel* es una función matemática que calcula el producto escalar entre dos vectores en un espacio de características de mayor dimensionalidad, sin necesidad de realizar la transformación explícita de los datos a dicho espacio. Esto permite encontrar un hiperplano separador en ese espacio de mayor dimensión de manera computacionalmente eficiente.

**Kernels más comunes:**
- **Lineal (`linear`):** No aplica ninguna transformación. Es eficiente y recomendable como punto de partida si se sospecha que la relación es lineal.
- **Polinomial (`poly`):** Modela interacciones polinómicas. Sus hiperparámetros incluyen el grado del polinomio (`degree`).
- **Base Radial (RBF - `rbf`):** Es el kernel más utilizado y potente por su capacidad para manejar relaciones no lineales complejas. Su hiperparámetro `gamma` define la influencia de un único ejemplo de entrenamiento. Un `gamma` alto puede llevar a sobreajuste, mientras que un `gamma` bajo puede resultar en subajuste.

In [ ]:
# Importaciones estándar y configuración del entorno
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_curve, auc, RocCurveDisplay
)
from sklearn.datasets import make_moons, load_iris
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
import warnings

# Configuraciones generales
warnings.filterwarnings('ignore')
np.random.seed(42)
%matplotlib inline
sns.set_theme(style="whitegrid")

## Ejemplo Práctico: Clasificación SVM con el Dataset Titanic

Se aplicará un clasificador SVM para predecir la supervivencia de los pasajeros del Titanic. Para ello, se seguirá un flujo de trabajo estructurado que incluye:
1. Carga y preprocesamiento de datos.
2. Construcción de un `Pipeline` para encapsular la transformación de datos y el modelo.
3. Búsqueda de hiperparámetros óptimos (`C` y `gamma`) mediante `GridSearchCV`.
4. Evaluación del rendimiento del modelo final con diversas métricas.

In [ ]:
# Carga y preparación del dataset Titanic
titanic = sns.load_dataset('titanic')

# Se seleccionan las características relevantes y se eliminan filas con valores nulos en 'embarked'
titanic = titanic[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']].copy()
titanic = titanic.dropna(subset=['embarked'])

# Definición de características (X) y variable objetivo (y)
X = titanic.drop(columns=['survived'])
y = titanic['survived']

# Identificación de tipos de columnas para el preprocesamiento
num_cols = ['age', 'sibsp', 'parch', 'fare']
cat_cols = ['pclass', 'sex', 'embarked']

# División en conjuntos de entrenamiento y prueba (train/test split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

In [ ]:
# Creación del pipeline de preprocesamiento para variables numéricas y categóricas

# Pipeline para datos numéricos: imputación de mediana y escalado estándar
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para datos categóricos: imputación de moda y codificación one-hot
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinación de ambos pipelines en un único ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# Creación del pipeline final que integra el preprocesador y el clasificador SVM
pipe = Pipeline([
    ('pre', preprocessor),
    ('clf', SVC(kernel='rbf', probability=True, random_state=42))
])

In [ ]:
# Búsqueda en grilla (Grid Search) para los hiperparámetros C y gamma
param_grid = {
    'clf__C': [0.1, 1, 10, 100],
    'clf__gamma': ['scale', 0.01, 0.1, 1]
}

# Se utiliza validación cruzada (cv=5) y se optimiza para el área bajo la curva ROC (roc_auc)
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='roc_auc', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

print(f'Mejores parámetros encontrados: {grid.best_params_}')
print(f'Mejor ROC AUC en validación cruzada: {grid.best_score_:.4f}')

In [ ]:
# Evaluación del mejor modelo en el conjunto de prueba
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# Métricas de rendimiento
print(f'Exactitud (Accuracy) en prueba: {(y_pred == y_test).mean():.4f}')
print('\nReporte de Clasificación:\n', classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.show()

In [ ]:
# Visualización de la Curva ROC y cálculo del AUC
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

print(f'Área Bajo la Curva (AUC) en prueba: {roc_auc:.4f}')

# Gráfico de la curva ROC
RocCurveDisplay.from_estimator(best_model, X_test, y_test)
plt.title('Curva ROC (Receiver Operating Characteristic)')
plt.plot([0, 1], [0, 1], 'r--', label='Clasificador Aleatorio')
plt.legend()
plt.show()

## Visualización de Fronteras de Decisión

Para una comprensión más intuitiva de cómo funcionan los diferentes kernels y el hiperparámetro `C`, se utilizará un conjunto de datos sintético en 2D. Esto permite visualizar directamente el hiperplano y los márgenes que el modelo aprende.

In [ ]:
# Creación de un dataset sintético no linealmente separable
X_synth, y_synth = make_moons(n_samples=100, noise=0.15, random_state=42)
X_synth_scaled = StandardScaler().fit_transform(X_synth)

# Función auxiliar para visualizar la frontera de decisión
def plot_decision_boundary(clf, X, y, title):
    h = .02
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm, edgecolors='k')
    plt.title(title)
    plt.xlabel('Característica 1 (escalada)')
    plt.ylabel('Característica 2 (escalada)')
    plt.show()

# Entrenamiento y visualización de diferentes modelos SVM
svm_linear = SVC(kernel='linear', C=1).fit(X_synth_scaled, y_synth)
plot_decision_boundary(svm_linear, X_synth_scaled, y_synth, 'SVM con Kernel Lineal (C=1)')

svm_rbf_low_c = SVC(kernel='rbf', C=0.1, gamma='auto').fit(X_synth_scaled, y_synth)
plot_decision_boundary(svm_rbf_low_c, X_synth_scaled, y_synth, 'SVM con Kernel RBF (C=0.1, Margen Suave)')

svm_rbf_high_c = SVC(kernel='rbf', C=100, gamma='auto').fit(X_synth_scaled, y_synth)
plot_decision_boundary(svm_rbf_high_c, X_synth_scaled, y_synth, 'SVM con Kernel RBF (C=100, Margen Duro)')

## Ejemplo de Clustering: Método del Codo y Silueta con el Dataset Iris

Aunque las SVM son principalmente para clasificación, el análisis de clustering es una técnica fundamental en el aprendizaje no supervisado. Se utilizarán los métodos del **Codo** y de la **Silueta** para determinar el número óptimo de clusters en el clásico dataset Iris.

In [ ]:
# Carga del dataset Iris
iris = load_iris()
X_iris = iris.data

# Método del Codo para encontrar el número óptimo de clusters (k)
inertia = []
k_range = range(1, 11)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_iris)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 6))
plt.plot(k_range, inertia, marker='o')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo para Selección de k')
plt.xticks(k_range)
plt.show()

In [ ]:
# Análisis de Silueta para k=3 (valor sugerido por el método del codo)
best_k = 3
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_iris)

silhouette_avg = silhouette_score(X_iris, labels)
print(f'Para k = {best_k}, el puntaje de silueta promedio es: {silhouette_avg:.4f}')

# Gráfico de Silueta
silhouette_vals = silhouette_samples(X_iris, labels)

plt.figure(figsize=(8, 6))
y_lower = 10
for i in range(best_k):
    ith_cluster_sil_vals = silhouette_vals[labels == i]
    ith_cluster_sil_vals.sort()
    size_cluster_i = ith_cluster_sil_vals.shape[0]
    y_upper = y_lower + size_cluster_i
    plt.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_vals)
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10

plt.axvline(x=silhouette_avg, color='red', linestyle='--')
plt.xlabel('Valores del coeficiente de Silueta')
plt.ylabel('Etiqueta del Cluster')
plt.title(f'Gráfico de Silueta para k = {best_k}')
plt.show()

## Resumen y Sugerencias

- Este cuaderno ha cubierto los conceptos fundamentales de SVM, incluyendo un ejemplo práctico de clasificación con un pipeline limpio, búsqueda de hiperparámetros, evaluación con ROC/AUC y visualización de fronteras de decisión.
- Se ha demostrado el poder de los pipelines de Scikit-learn para crear flujos de trabajo de machine learning robustos y reproducibles.
- Para clustering, se han mostrado los métodos del Codo y de la Silueta como herramientas para la selección del número óptimo de clusters, siendo el análisis de silueta a menudo más informativo.

**Sugerencias para extender el análisis:**
- Implementar curvas ROC validadas cruzadamente (con `StratifiedKFold`).
- Probar diferentes kernels (ej. `poly`) y visualizar el efecto en los márgenes.
- Para problemas de clasificación multiclase, utilizar estrategias como One-vs-Rest y promediado micro/macro de las métricas.